In [1]:
import pyshark
import re
from pprint import pprint
path_to_file = 'project2.trace'

In [7]:
#Question 1
filtered_cap = pyshark.FileCapture(path_to_file, display_filter='http.response')
filtered_cap_list = list(filtered_cap)
print "number of packets: " + str(len(filtered_cap_list))

number of packets: 170


In [12]:
ip_dict = dict()
for packet in filtered_cap_list:
    if packet.ip:
        ip_addr = packet.ip.src
        if ip_addr in ip_dict:
            ip_dict[ip_addr]+=1
        else:
            ip_dict[ip_addr]=1
print "IP address(es) (" + str(len(ip_dict)) + "):"
for key in ip_dict:
    print key

IP address(es) (36):
155.111.186.252
40.187.57.142
93.119.134.44
155.231.237.70
159.79.22.249
159.70.229.173
35.183.215.204
205.232.201.218
93.199.112.45
33.247.152.113
142.165.192.188
136.93.4.213
143.138.66.97
97.145.19.119
44.111.85.82
159.79.22.198
205.234.49.157
154.87.109.177
44.131.48.102
33.247.152.101
159.79.22.194
154.87.109.40
248.78.109.66
142.165.192.177
34.30.235.180
154.27.68.55
111.4.186.50
143.138.4.147
37.199.226.67
37.120.175.85
205.232.203.30
138.59.102.27
143.179.11.189
251.235.172.148
44.131.51.48
44.131.51.161


In [8]:
#Question 2
filtered_cap_2 = pyshark.FileCapture(path_to_file, display_filter='http.request.full_uri contains "../../../"')

In [9]:
filtered_cap_list_2 = list(filtered_cap_2)
print "number of packets: " + str(len(filtered_cap_list_2))

number of packets: 3


In [11]:
ip_dict_2 = dict()
for packet in filtered_cap_list_2:
    if packet['ip']:
        ip_addr = packet['ip'].src
        if ip_addr in ip_dict_2:
            ip_dict_2[ip_addr]+=1
        else:
            ip_dict_2[ip_addr]=1
print "IP address(es) (" + str(len(ip_dict_2)) + "):"
for key in ip_dict_2:
    print key

IP address(es) (1):
42.9.203.117


In [13]:
#Question 3
filtered_cap_3 = pyshark.FileCapture(path_to_file, display_filter='ftp.request.command contains "USER" || ftp.request.command contains "PASS"')

In [14]:
filtered_cap_list_3 = list(filtered_cap_3)
print "number of packets: " + str(len(filtered_cap_list_3))

number of packets: 17


In [15]:
ip_dict_3 = dict()
for packet in filtered_cap_list_3:
    if packet['ip']:
        ip_addr = packet['ip'].src
        if ip_addr in ip_dict_3:
            ip_dict_3[ip_addr]+=1
        else:
            ip_dict_3[ip_addr]=1
print "IP address(es) (" + str(len(ip_dict_3)) + "):"
for key in ip_dict_3:
    print key, ip_dict_3[key]

IP address(es) (3):
172.27.37.232 2
251.215.184.138 2
248.35.162.92 13


In [18]:
#Question 4
string_part_1 = ' contains "USER" || '
string_part_2 = ' contains "PASS" ||'
string_part_3 = ' contains "username" ||'
string_part_4 = ' contains "password"'
protocol_strings = ["telnet", "ftp.request.command", "http.request.method == POST", "pop.request.command"]
filtered_cap_4_dict = dict()
for x in protocol_strings:
    filter_option = x+string_part_1+x+string_part_2+x+string_part_3+x+string_part_4
    #print filter_option
    temp_list = list(pyshark.FileCapture(path_to_file, display_filter=filter_option))
    filtered_cap_4_dict[x]=temp_list
for key in filtered_cap_4_dict:
    print key, len(filtered_cap_4_dict[key])

http.request.method == POST 0
pop.request.command 0
ftp.request.command 17
telnet 3


In [20]:
#script to check ftp first
"""
This function checks if a response packet is success or failure based on the sequence number of the
response packet, as well as lists of known success/fail response packets.
"""
def check_response(ack_seq, success_response_packets, fail_response_packets):
    for packet in success_response_packets:
        if packet.tcp.seq == ack_seq:
            return "success"
    for packet in fail_response_packets:
        if packet.tcp.seq == ack_seq:
            return "fail"
    return "error"

"""
This part identify successful login attempts and stores the source ip and stream number.
"""
ftp_success_response_packets_list = list(pyshark.FileCapture(path_to_file, display_filter='ftp.response.code == 230'))
ftp_fail_response_packets_list = list(pyshark.FileCapture(path_to_file, display_filter='ftp.response.code == 530'))
ftp_pass_packets_list = list(pyshark.FileCapture(path_to_file, display_filter='ftp.request.command contains "PASS"'))
ip_of_interest_list = []
streams_of_interest = []
for packet in ftp_pass_packets_list:
    stream = packet.tcp.stream
    #curr_seq = packet.tcp.seq
    ack_seq = packet.tcp.ack
    curr_src_ip = packet.ip.src
    if check_response(ack_seq, ftp_success_response_packets_list, ftp_fail_response_packets_list) == "success":
        ip_of_interest_list.append(curr_src_ip)
        streams_of_interest.append(stream)
print "IP addresses and TCP streams of interest:"
print ip_of_interest_list, streams_of_interest

IP addresses and TCP streams of interest:
['251.215.184.138', '172.27.37.232'] ['28', '55']


In [21]:
"""
This part pulls out the user and pass we're interested in.
"""
login_info = dict()
j = 0
for ip in ip_of_interest_list:
    arg_dict = dict()
    for packet in filtered_cap_4_dict["ftp.request.command"]:
        if str(packet.ip.src)==ip and str(packet.tcp.stream)==str(streams_of_interest[j]):
            temp_string = str(packet["ftp"])
            #print temp_string
            array_split = re.split(' |\n|\r|\t', temp_string)
            #print array_split
            i = 0
            arg_value, arg_type = "", ""
            for x in array_split:
                if x == "arg:":
                    arg_value = array_split[i+1]
                    #print arg_value
                if x == "command:":
                    arg_type = array_split[i+1]
                    #print arg_type
                i += 1
            arg_dict[arg_type] = arg_value
    login_info[ip] = arg_dict
    j+=1
print "Login info found:"
pprint(login_info)

Login info found:
{'172.27.37.232': {'PASS': 'thisissosecure', 'USER': 'calrules'},
 '251.215.184.138': {'PASS': '', 'USER': 'anonymous'}}


In [23]:
#Question 5
filtered_cap_5 = list(pyshark.FileCapture(path_to_file, display_filter='http.server contains "Apache/"'))
print "number of packets: " + str(len(filtered_cap_5))

number of packets: 103


In [24]:
def get_apache_ver(packet):
    apache_ver = ""
    txt = str(packet["http"])
    split_txt = txt.split(" ")
    for x in split_txt:
        if "Apache" in x:
            apache_ver = x
    return apache_ver

apache_ver_list = []
for packet in filtered_cap_5:
    apache_ver_list.append(get_apache_ver(packet))

sorted_filtered_cap_5 = [x for (y,x) in sorted(zip(apache_ver_list, filtered_cap_5))]
print "Server with oldest version:"
print sorted_filtered_cap_5[0].ip.src, sorted_filtered_cap_5[0].http.server

Server with oldest version:
205.232.201.218 Apache/1.3.28 (Debian GNU/Linux)


In [25]:
#Question 6
filtered_cap_6 = list(pyshark.FileCapture(path_to_file, display_filter='dns.flags == 0x0100')) #recursive DNS
print "number of packets: " + str(len(filtered_cap_6))

number of packets: 418


In [26]:
ip_to_srcport = dict()
for packet in filtered_cap_6:
    ip_addr = str(packet.ip.src)
    #print ip_addr
    src_port = str(packet.udp.port)
    #print src_port
    if ip_addr not in ip_to_srcport:
        ip_to_srcport[ip_addr] = []
        #print ip_to_srcport[ip_addr]
        ip_to_srcport[ip_addr].append(src_port)
        #print ip_to_srcport[ip_addr]
    else:
        for port in ip_to_srcport[ip_addr]:
            if src_port != port:
                ip_to_srcport[ip_addr].append(src_port)
                #print ip_to_srcport[ip_addr]
                break
print "IP addresses of DNS resolvers without port randomization:"
for ip in ip_to_srcport:
    port_list = ip_to_srcport[ip]
    if len(port_list) == 1:
        print ip

IP addresses of DNS resolvers without port randomization:
205.232.201.218
205.232.201.195


In [27]:
#Question 7
#0x02 = 000000010 2nd bit = SYN = sync seq no = only the first packet sent from each end should have this flag set
filtered_cap_7 = list(pyshark.FileCapture(path_to_file, display_filter='tcp.flags == 0x02'))
print "number of packets: " + str(len(filtered_cap_7))

number of packets: 108


In [63]:
ip_src_to_seq_nums = dict()
for packet in filtered_cap_7:
    src_ip = packet.ip.src
    #dst_ip = packet.ip.dst
    seq = packet.tcp.seq
    #ip_pair = src_ip + " " + dst_ip
    if src_ip not in ip_src_to_seq_nums:
        ip_src_to_seq_nums[src_ip] = []
    ip_src_to_seq_nums[src_ip].append(seq)
#pprint(ip_src_to_seq_nums)

In [47]:
ip_src_to_seq_range = dict()
for ip in ip_src_to_seq_nums:
    seq_range = 0
    if len(ip_src_to_seq_nums[ip]) <= 4:
        ip_src_to_seq_range[ip] = -1
    else:
        highest = 0
        lowest = 0
        for seq_num in ip_src_to_seq_nums[ip]:
            if int(seq_num) > highest:
                highest = int(seq_num)
        lowest = highest
        for seq_num in ip_src_to_seq_nums[ip]:
            if int(seq_num) < lowest:
                lowest = int(seq_num)
        seq_range = highest - lowest
        ip_src_to_seq_range[ip] = str(seq_range)
        #print ip, highest, lowest, seq_range

ip_src_list = []
ip_range_list = []
for ip in ip_src_to_seq_range:
    ip_src_list.append(str(ip))
    ip_range_list.append(int(ip_src_to_seq_range[ip]))
sorted_ip_src_list = [x for (y,x) in sorted(zip(ip_range_list, ip_src_list))]
sorted_range_list = sorted(ip_range_list)
print "Top 2 IP addresses with most random seq number ranges:"
print sorted_ip_src_list[-1], sorted_range_list[-1]
print sorted_ip_src_list[-2], sorted_range_list[-2]

Top 2 IP addresses with most random seq number ranges:
251.215.153.250 3890239477
205.232.201.195 243605121


In [50]:
#Question 8
filtered_cap_8_list_dict = dict()
ttl_list = []
for x in xrange(1,6):
    ttl_list.append(x)
for ttl in ttl_list:
    query = 'udp && ip.ttl == ' + str(ttl)
    filtered_cap_8_list_dict[ttl] = list(pyshark.FileCapture(path_to_file, display_filter=query))
    print "number of packets: " + str(len(filtered_cap_8_list_dict[ttl])) + " (TTL: " + str(ttl) +")"

number of packets: 355 (TTL: 1)
number of packets: 30 (TTL: 2)
number of packets: 30 (TTL: 3)
number of packets: 30 (TTL: 4)
number of packets: 30 (TTL: 5)


In [54]:
ip_to_ttl_increments = dict()
for ttl in filtered_cap_8_list_dict:
    for packet in filtered_cap_8_list_dict[ttl]:
        ip_src = packet.ip.src
        ip_dst = packet.ip.dst
        ip_pair = ip_src + " " + ip_dst
        if ip_pair not in ip_to_ttl_increments:
            ttl_counts = dict()
            ip_to_ttl_increments[ip_pair] = ttl_counts
        if ttl not in ip_to_ttl_increments[ip_pair]:
            ip_to_ttl_increments[ip_pair][ttl] = 0
        ip_to_ttl_increments[ip_pair][ttl]+=1
print "IP addresses and number of packets containing TTLs 1 to 5:"
pprint(ip_to_ttl_increments)

IP addresses and number of packets containing TTLs 1 to 5:
{'172.18.74.109 172.25.130.244': {1: 2},
 '172.29.115.68 176.95.3.38': {1: 1},
 '172.31.250.21 172.25.130.244': {1: 2},
 '205.232.201.10 188.183.25.203': {1: 155},
 '205.232.201.11 188.183.25.203': {1: 165},
 '248.86.240.130 159.79.23.208': {1: 30, 2: 30, 3: 30, 4: 30, 5: 30}}


In [56]:
#Question 9
filtered_cap_9_list = list(pyshark.FileCapture(path_to_file, display_filter='http.request.full_uri contains "%3Cscript%3E" || http.request.full_uri contains "<script>"'))
print "number of packets: " + str(len(filtered_cap_9_list))

number of packets: 18


In [57]:
streams_to_ip = dict()
for packet in filtered_cap_9_list:
    ip_pair = packet.ip.src + " " + packet.ip.dst
    tcp_stream = packet.tcp.stream
    get_req = str(packet.http.chat)
    if get_req.find("%3Cscript%3E") != -1:
        index_start = get_req.find("%3Cscript%3E")
        index_end = get_req.find("HTTP/")-1
        script_string = get_req[index_start:index_end]
    elif get_req.find("<script>") != -1:
        index_start = get_req.find("<script>")
        index_end = get_req.find("HTTP/")-1
        script_string = get_req[index_start:index_end]
    else:
        script_string = "none!"
        print "error: no script string for stream " + tcp_stream
    streams_to_ip[tcp_stream] = [ip_pair, script_string]

stream_http_response_ok_list = []
for tcp_stream in streams_to_ip:
    query = 'tcp.stream eq ' + tcp_stream + ' && http.response.code == 200'
    stream_cap = list(pyshark.FileCapture(path_to_file, display_filter=query))
    if stream_cap:
        stream_http_response_ok_list.append(stream_cap)

In [64]:
"""
This function checks number of times a particular XSS appears in page source code.
"""
def check_stored_xss(packet, script_string):
    page_data = str(packet['data-text-lines'])
    if script_string[0:1]=="%":
        script_string = script_string.replace("%3Cscript%3E", "")
        script_string = script_string.replace("%3C/script%3E", "")
        print script_string
    elif script_string[0:1]=="<":
        script_string = script_string.replace("<script>", "")
        script_string = script_string.replace("</script>", "")
        script_string = script_string.replace("</script", "")
        print script_string
    instances = page_data.count(script_string)
    return instances

print "Script contents with list index, ip server/client, http response, number of times the script appears in html:"
index = 0
for stream_cap in stream_http_response_ok_list:
    if len(stream_cap) == 1:
        ip_pair = stream_cap[0].ip.src + " " + stream_cap[0].ip.dst
        stream = stream_cap[0].tcp.stream
        script_string = streams_to_ip[stream][1]
        instances = check_stored_xss(stream_cap[0], script_string)
        print "index " + str(index) + ", " + ip_pair + ", " + "http OK, " + str(instances)
    else:
        print "error: more than one http ok packet"
    index += 1

Script contents with list index, ip server/client, http response, number of times the script appears in html:
alert('struts_sa_surl_xss.nasl')
index 0, 159.79.22.249 251.215.153.250, http OK, 0
alert(123456789)
index 1, 44.111.85.82 251.215.76.253, http OK, 11
document.cookie=%22testhzlg=9267;%22
index 2, 159.79.22.249 251.215.153.250, http OK, 0
alert('struts_sa_surl_xss.nasl')
index 3, 159.79.22.249 251.215.153.250, http OK, 0
alert('struts_sa_surl_xss.nasl')
index 4, 159.79.22.249 251.215.153.250, http OK, 0
foo
index 5, 159.79.22.249 251.215.153.250, http OK, 0
cross_site_scripting.nasl
index 6, 159.79.22.249 251.215.153.250, http OK, 0
